# End to End CLV Model Notebook

# Getting Started with Snowflake Feature Store
We will use the Use-Case to show how Snowflake Feature Store (and Model Registry) can be used to maintain & store features, retrieve them for training and perform micro-batch inference.

In the development (TRAINING) enviroment we will 
- create FeatureViews in the Feature Store that maintain the required customer-behaviour features.
- use these Features to train a model, and save the model in the Snowflake model-registry.
- plot the clusters for the trained model to visually verify. 

In the production (SERVING) environment we will
- re-create the FeatureViews on production data
- generate an Inference FeatureView that uses the saved model to perform incremental inference

# Feature Engineering & Model Training

In [ ]:
%load_ext autoreload
%autoreload 2

#### Notebook Packages

In [ ]:
# Python packages
import os
import json
import timeit

# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec

import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Snowflake Feature Store
from snowflake.ml.feature_store import (
    FeatureView,
    Entity
)

# COMMON FUNCTIONS
from helper.useful_fns import dataset_check_and_update, formatSQL, create_ModelRegistry, create_FeatureStore, create_SF_Session, get_spine_df

### Setup Snowflake connection and database parameters

In [ ]:
role_name, database_name, schema_name, session, warehouse_name = create_SF_Session(connection_file = '../../connection.json')

## MODEL DEVELOPMENT
* Create Snowflake Model-Registry
* Create Snowflake Feature-Store
* Establish and Create CUSTOMER Entity in the development Snowflake FeatureStore
* Create Source Data references and perform basic data-cleansing
* Create & Run Preprocessing Function to create features
* Create FeatureView_Preprocess from Preprocess Dataframe SQL
* Create training data from FeatureView_Preprocess (asof join)
* Create & Fit Snowpark-ml pipeline 
* Save model in Model Registry
* 'Verify' and approve model
* Create new FeatureView_Model_Inference with Transforms UDF + KMeans model

In [ ]:
# Create/Reference Snowflake Model Registry - Common across Environments
mr = create_ModelRegistry(session, database_name, '_MODELLING')

# Create/Reference Snowflake Feature Store - Common across Environments
fs = create_FeatureStore(session, database_name, '_FEATURE_STORE', warehouse_name)


In [ ]:
cust_tbl = '.'.join([database_name, schema_name,'CUSTOMERS'])
cust_sdf = session.table(cust_tbl)
print(cust_tbl, cust_sdf.count())
cust_sdf.limit(10).show()

### CUSTOMER Entity
Establish and Create CUSTOMER Entity in Snowflake FeatureStore for this Use-Case

In [ ]:
if "CUSTOMER" not in json.loads(fs.list_entities().select(F.to_json(F.array_agg("NAME", True))).collect()[0][0]):
    customer_entity = Entity(name="CUSTOMER", join_keys=["CUSTOMER_ID"],desc="Primary Key for CUSTOMER ORDER")
    fs.register_entity(customer_entity)
else:
    customer_entity = fs.get_entity("CUSTOMER")

fs.list_entities().show()

 ### Create & Load Source Data

Our Feature engineering pipelines are defined using Snowpark dataframes (or SQL expressions).  In the `QS_feature_engineering_fns.py` file we have created two feature engineering functions to create our pipeline :
* __uc01_load_data__(order_data: DataFrame, lineitem_data: DataFrame, order_returns_data: DataFrame) -> DataFrame   
* __uc01_pre_process__(data: DataFrame) -> DataFrame

`uc01_load_data`, takes the source tables, as dataframe objects, and joins them together, performing some data-cleansing by replacing NA's with default values. It returns a dataframe as it's output.

`uc01_pre_process`, takes the dataframe output from `uc01_load_data`  and performs aggregation on it to derive some features that will be used in our segmentation model.  It returns a dataframe as output, which we will use to provide the feature-pipeline definition within our FeatureView.

In this way we can build up a complex pipeline step-by-step and use it to derive a FeatureView, that will be maintained as a pipeline in Snowflake.

We will import the functions, and create dataframes from them using the dataframes we created earlier pointing to the tables in our TRAINING (Development) schema.  We will use the last dataframe we create at the end of the pipeline as our input to the FeatureView.


In [ ]:
# Feature Engineering Functions
from feature_engineering_fns import uc01_load_data, uc01_pre_process

In [ ]:
# Tables
cust_tbl                    = '.'.join([database_name, schema_name,'CUSTOMERS'])
behavior_tbl                = '.'.join([database_name, schema_name,'PURCHASE_BEHAVIOR'])

# Snowpark Dataframe
cust_sdf              = session.table(cust_tbl)
behavior_tbl          = session.table(behavior_tbl)

# Row Counts
print(f'''\nTABLE ROW_COUNTS IN {schema_name}''')
print(cust_tbl, cust_sdf.count())
print(behavior_tbl, behavior_tbl.count())

In [ ]:
raw_data = uc01_load_data(cust_sdf, behavior_tbl)

In [ ]:
# Format and print the SQL for the Snowpark Dataframe
rd_sql = formatSQL(raw_data.queries['queries'][0], True)
print(os.linesep.join(rd_sql.split(os.linesep)[:1000]))

In [ ]:
raw_data.show()

### Create & Run Preprocessing Function 

In [ ]:
preprocessed_data = uc01_pre_process(raw_data)

In [ ]:
preprocessed_data.show()

In [ ]:
# Format and print the SQL for the Snowpark Dataframe
ppd_sql = formatSQL(preprocessed_data.queries['queries'][0], True)
print(os.linesep.join(ppd_sql.split(os.linesep)[:1000]))

### Create Preprocessing FeatureView from Preprocess Dataframe (SQL)

In [ ]:
# Define descriptions for the FeatureView's Features.  These will be added as comments to the database object
preprocess_features_desc = {  
   "AVERAGE_ORDER_PER_MONTH":"Average number of orders per month",
   "DAYS_SINCE_LAST_PURCHASE":"Days since last purchase",
   "DAYS_SINCE_SIGNUP":"Days since signup",
   "EXPECTED_DAYS_BETWEEN_PURCHASES":"Expected days between purchases",
   "DAYS_SINCE_EXPECTED_LAST_PURCHASE_DATE":"Days since expected last purchase date from LAST_PURCHASE_DATE"
}

ppd_fv_name    = "FV_PREPROCESS"
ppd_fv_version = "V_1"

try:
   # If FeatureView already exists just return the reference to it
   fv_uc01_preprocess = fs.get_feature_view(name=ppd_fv_name,version=ppd_fv_version)
except:
   # Create the FeatureView instance
   fv_uc01_preprocess_instance = FeatureView(
      name=ppd_fv_name, 
      entities=[customer_entity], 
      feature_df=preprocessed_data,      # <- We can use the snowpark dataframe as-is from our Python
      # feature_df=preprocessed_data.queries['queries'][0],    # <- Or we can use SQL, in this case linted from the dataframe generated SQL to make more human readable
      timestamp_col="BEHAVIOR_UPDATED_AT",
      refresh_freq="60 minute",            # <- specifying optional refresh_freq creates FeatureView as Dynamic Table, else created as View.
      desc="Customer Modelling Features").attach_feature_desc(preprocess_features_desc)

   # Register the FeatureView instance.  Creates  object in Snowflake
   fv_uc01_preprocess = fs.register_feature_view(
      feature_view=fv_uc01_preprocess_instance, 
      version=ppd_fv_version, 
      block=True,     # whether function call blocks until initial data is available
      overwrite=False # whether to replace existing feature view with same name/version
   )
   print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} created")   
else:
   print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} already created")
finally:
   fs.list_feature_views().show(20)
spine = fv_uc01_preprocess

In [ ]:
# You can also use the following to retrieve a Feature View instance for use within Python
FV_UC01_PREPROCESS_V_1 = fs.get_feature_view(ppd_fv_name, 'V_1')

In [ ]:
# We can look at the FeatureView's contents with
FV_UC01_PREPROCESS_V_1.feature_df.sort(F.col("CUSTOMER_ID"), ascending=False).show(10)

### Create training data Dataset from FeatureView_Preprocess

In [ ]:
# Create Spine
spine_sdf = get_spine_df(spine)
spine_sdf.sort('CUSTOMER_ID').show(5)

In [ ]:
from helper.useful_fns import dataset_check_and_update
def generate_training_df(spine_sdf, feature_view, feature_store):
    dataset_name = 'TRAINING_DATASET'
    schema_name = feature_store.list_feature_views().to_pandas()['SCHEMA_NAME'][0]

    dataset_version = dataset_check_and_update(session, dataset_name, schema_name= schema_name)
    # Generate_Dataset
    training_dataset = feature_store.generate_dataset( 
        name = dataset_name,
        version = dataset_version,
        spine_df = spine_sdf, 
        features = [feature_view], 
        spine_timestamp_col = 'ASOF_DATE'
        )                                     
    # Create a snowpark dataframe reference from the Dataset
    training_dataset_sdf = training_dataset.read.to_snowpark_dataframe()
    
    return training_dataset_sdf

In [ ]:
# Generate_Dataset
training_dataset_sdf_v1 = generate_training_df(spine_sdf, fv_uc01_preprocess, feature_store=fs)
# Display some sample data
training_dataset_sdf_v1.sort('CUSTOMER_ID').show(5)

In [ ]:
training_dataset_sdf_v1.to_pandas().head()

## CLEAN UP

In [ ]:
# session.close()

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
formatted_time = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%A, %B %d, %Y %I:%M:%S %p %Z")

print(f"The last run time in Melbourne is: {formatted_time}")